## Recreating the following for classification problem
https://www.tensorflow.org/text/tutorials/transformer

Need tensorflow-text for using the BERT model tokenizer

In [1]:
# Install the nightly version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q -U tensorflow-text tensorflow

     |████████████████████████████████| 5.9 MB 7.9 MB/s 
     |████████████████████████████████| 578.0 MB 13 kB/s 
     |████████████████████████████████| 438 kB 64.6 MB/s 
     |████████████████████████████████| 5.9 MB 47.0 MB/s 
     |████████████████████████████████| 1.7 MB 44.2 MB/s 


Import the necessary modules:

In [115]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text
from sklearn.utils import shuffle 

Doing necessary data preprocessing

In [116]:
os.listdir('.')

['.config',
 'ted_hrlr_translate_pt_en_converter',
 'twitter_validation.csv',
 'ted_hrlr_translate_pt_en_converter.zip',
 'twitter_training.csv',
 'sample_data']

In [117]:
training_data_loc = "./twitter_training.csv"
val_data_loc = "./twitter_validation.csv"

In [118]:
df = pd.read_csv(training_data_loc)
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [119]:
df = df[df.columns[2:]]

In [120]:
df = df.rename(columns={"Positive":"Sentiment", "im getting on borderlands and i will murder you all ,":"sentence"})

In [121]:
df['Sentiment'].value_counts()

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [122]:
df = df[df.apply(lambda x: (x["Sentiment"] == "Positive") or (x["Sentiment"] == "Negative"), axis=1)]

In [123]:
df['Sentiment'].value_counts()

Negative    22542
Positive    20831
Name: Sentiment, dtype: int64

In [124]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

In [125]:
df = shuffle(df)

df

,Sentiment,sentence
4221,1,AND DON ’ t yet be FUCKEN LATE
47473,1,I love that my Home Depot management admires w...
62865,1,5 Games I Love.. Phoenix 1. Series Call of Dut...
53236,1,The two things that have made me grieve the mo...
60200,1,Still my Gmail account.
...,...,...
20626,0,All to
2050,0,Live-action pirate film is just a really lame ...
22924,0,I
16763,1,More beautiful than most girls i know..


In [126]:
df.isna().value_counts()

Sentiment  sentence
False      False       43012
           True          361
dtype: int64

In [127]:
df.dropna(inplace=True) # dropping empty sentences

In [128]:
df

,Sentiment,sentence
4221,1,AND DON ’ t yet be FUCKEN LATE
47473,1,I love that my Home Depot management admires w...
62865,1,5 Games I Love.. Phoenix 1. Series Call of Dut...
53236,1,The two things that have made me grieve the mo...
60200,1,Still my Gmail account.
...,...,...
20626,0,All to
2050,0,Live-action pirate film is just a really lame ...
22924,0,I
16763,1,More beautiful than most girls i know..


In [130]:
labels = df['Sentiment']
features = df['sentence']

## Tokenizer
using subword tokenizer implementation from Bert model

In [131]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

'./ted_hrlr_translate_pt_en_converter.zip'

In [132]:
tokenizers = tf.saved_model.load(model_name)

In [133]:
[item for item in dir(tokenizers) if not item.startswith('_')]

['en',
 'graph_debug_info',
 'pt',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version']

The `tf.saved_model` contains two text tokenizers, one for English and one for Portuguese. We just need the English one

In [20]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [134]:
sample = features.sample(n=5)
sample

74012    AMD cannot save my gaming. No one buys Amd car...
64742    @EAMaddenNFL  please fix your bugs. They are a...
71343                        This year is how much I enjoy
7227                 Suddenly I am unable to actually swim
39542    Perfect Hearthstone Draft – youtu.be/cntGVPcna...
Name: sentence, dtype: object

In [135]:
text_sample = sample.iloc[0]
print(text_sample)

AMD cannot save my gaming. No one buys Amd cards in the main stream. ". Intel 1070 Ti out sold 80 200 GTX 1060 out sold RX580/480.. GTX 780/780ti out sold R9 290/290x. M GTX 2060/2060s out sold Rx 5700/5700xt... Jensen about fuck your wallet.


In [136]:
encoded = tokenizers.en.tokenize(sample)
# print(encoded)
print('Padded and encoded embeddings:')
for row in encoded.to_list():
  print(row)

Padded and encoded embeddings:
[2, 340, 178, 94, 6492, 864, 99, 43, 2199, 138, 15, 154, 103, 872, 88, 340, 178, 1703, 77, 71, 1186, 3694, 15, 1, 15, 77, 5559, 308, 4278, 56, 388, 122, 2190, 1242, 1176, 43, 271, 1394, 308, 5911, 122, 2190, 54, 1394, 1800, 1164, 1010, 16, 6584, 1010, 15, 15, 43, 271, 1394, 24, 1164, 1010, 16, 24, 1164, 1010, 3746, 122, 2190, 54, 1374, 19, 1374, 1010, 16, 19, 1374, 1010, 1394, 15, 49, 43, 271, 1394, 447, 5911, 16, 447, 5911, 88, 122, 2190, 54, 1394, 22, 4278, 1010, 16, 22, 4278, 1010, 1394, 271, 15, 15, 15, 46, 1846, 840, 95, 42, 2713, 135, 747, 743, 15, 3]
[2, 31, 41, 2489, 5934, 269, 1192, 392, 1102, 1381, 135, 6080, 88, 15, 83, 86, 3090, 15, 426, 38, 3178, 627, 198, 22, 1661, 80, 140, 294, 73, 114, 15, 3]
[2, 81, 213, 80, 119, 170, 45, 2023, 3]
[2, 992, 45, 340, 6363, 408, 73, 150, 3392, 3]
[2, 937, 649, 4097, 5561, 2063, 3893, 271, 66, 79, 271, 795, 15, 97, 16, 39, 677, 651, 2204, 601, 787, 1225, 684, 1325, 3712, 31, 2237, 3]


The padded 2 and 3 stands for [START] and [END]

In [137]:
encoded[:, :128]

<tf.RaggedTensor [[2, 340, 178, 94, 6492, 864, 99, 43, 2199, 138, 15, 154, 103, 872, 88, 340,
  178, 1703, 77, 71, 1186, 3694, 15, 1, 15, 77, 5559, 308, 4278, 56, 388,
  122, 2190, 1242, 1176, 43, 271, 1394, 308, 5911, 122, 2190, 54, 1394,
  1800, 1164, 1010, 16, 6584, 1010, 15, 15, 43, 271, 1394, 24, 1164, 1010,
  16, 24, 1164, 1010, 3746, 122, 2190, 54, 1374, 19, 1374, 1010, 16, 19,
  1374, 1010, 1394, 15, 49, 43, 271, 1394, 447, 5911, 16, 447, 5911, 88,
  122, 2190, 54, 1394, 22, 4278, 1010, 16, 22, 4278, 1010, 1394, 271, 15,
  15, 15, 46, 1846, 840, 95, 42, 2713, 135, 747, 743, 15, 3]                ,
 [2, 31, 41, 2489, 5934, 269, 1192, 392, 1102, 1381, 135, 6080, 88, 15, 83,
  86, 3090, 15, 426, 38, 3178, 627, 198, 22, 1661, 80, 140, 294, 73, 114,
  15, 3]                                                                   ,
 [2, 81, 213, 80, 119, 170, 45, 2023, 3],
 [2, 992, 45, 340, 6363, 408, 73, 150, 3392, 3],
 [2, 937, 649, 4097, 5561, 2063, 3893, 271, 66, 79, 271, 795, 15, 97,

In [138]:
round_trip = tokenizers.en.detokenize(encoded[:, :128])

print('Original text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

Original text:
amd cannot save my gaming . no one buys amd cards in the main stream . [UNK] . intel 1070 ti out sold 80 200 gtx 1060 out sold rx580 / 480 . . gtx 780 / 780ti out sold r9 290 / 290x . m gtx 2060 / 2060s out sold rx 5700 / 5700xt . . . jensen about fuck your wallet .
@ eamaddennfl please fix your bugs . they are awful . having booted every 5 seconds is really old to me .
this year is how much i enjoy
suddenly i am unable to actually swim
perfect hearthstone draft – youtu . be / cntgvpcnabw via @ youtube


## Set up a data pipeline with `tf.data`

The following function takes batches of text as input, and converts them to a format suitable for training. 

1. It tokenizes them into ragged batches.
2. It trims each to be no longer than `MAX_TOKENS`.
3. It splits the output (English) tokens into inputs and labels. THese shifted by one step so that the `label` at each location is the id of the next token.
4. It converts the `RaggedTensor`s to padded dense `Tensor`s.
5. It returns an `(inputs, labels)` pair.

NOTE: We will bypass this method in the batch making area and utilize our own method since we are not using `tf.data`

# **Tokenizer**
Cannot prepare batches as required. I don't know how to fix it.

In [139]:
MAX_TOKENS=128
def prepare_batch(en): # not actually preparing batches
    en = tokenizers.en.tokenize(en)
    en = en[:, :128] # pads with `.` if the sentence is less than 128 words and strips if more than 128 words
    # print(en)
    # en_inputs = en[:, :-1].to_tensor()
    return en

In [140]:
tokenized_sentences = prepare_batch(features) # batch_size, 128

In [219]:
tokenized_sentences.to_tensor()

<tf.Tensor: shape=(43012, 128), dtype=int64, numpy=
array([[   2,   72,   91, ...,    0,    0,    0],
       [   2,   45,  302, ...,    0,    0,    0],
       [   2,   22, 2663, ...,    0,    0,    0],
       ...,
       [   2,   45,    3, ...,    0,    0,    0],
       [   2,  121,  595, ...,    0,    0,    0],
       [   2,   99,  582, ...,    0,    0,    0]])>

## Copied code 
we need to change it up a bit so we can use same definitions for our own classification problem

### Define the Embedding and Positional encoding

In [142]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [143]:
temp_encoding = positional_encoding(length=2048, depth=512)
temp_encoding

<tf.Tensor: shape=(2048, 512), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.84147096,  0.8218562 ,  0.8019618 , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.9092974 ,  0.9364147 ,  0.95814437, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 0.17589758, -0.18608274, -0.7070546 , ...,  0.9741639 ,
         0.97595036,  0.97761387],
       [-0.7333133 ,  0.7014913 ,  0.1447375 , ...,  0.9741387 ,
         0.97592694,  0.97759205],
       [-0.9683193 ,  0.98535496,  0.8799798 , ...,  0.9741135 ,
         0.9759035 ,  0.9775702 ]], dtype=float32)>

In [144]:
temp_encoding[tf.newaxis, :128, :].shape

TensorShape([1, 128, 512])

In [145]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x # (batch_size, length-of-signal, 512)


In [146]:
tokenizers.en.get_vocab_size()

<tf.Tensor: shape=(), dtype=int32, numpy=7010>

In [147]:
temp_layer = tf.keras.layers.Embedding(7010, 512, mask_zero=True)
# help(temp_layer)

In [148]:
temp_model = tf.keras.Sequential()
temp_layer = tf.keras.layers.Embedding(1000, 64, input_length=10, mask_zero=True)
temp_model.add(temp_layer)
# The model will take as input an integer matrix of size (batch,
# input_length), and the largest integer (i.e. word index) in the input
# should be no larger than 999 (vocabulary size).
# Now model.output_shape is (None, 10, 64), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(1, 10))
temp_model.compile('rmsprop', 'mse')
output_array = temp_model.predict(input_array)
# print(output_array)
print(output_array.shape)
#(1, 10, 64)

1/1 [==============================] - 0s 89ms/step
(1, 10, 64)


In [ ]:
# print(temp_layer.compute_mask(input_array, mask_zero=True)) # would not work as this is just for forwarding the compute_mask

In [149]:
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)
embed_en

In [ ]:
encoded.to_tensor()

In [151]:
# This is where the error pops up. I assume it is because of the way we are preparing batches.

# embed_en(encoded)
embed_en(encoded.to_tensor()).shape

TensorShape([5, 113, 512])

### Define the feed forward network

Define a function for the point-wise feed-forward network that you'll use later.

The network consists of two linear layers (`tf.keras.layers.Dense`) with a ReLU activation in-between:

In [152]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
  ])

### Define the encoder layer

In [153]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
    super().__init__()


    # Multi-head self-attention.
    self.mha = tf.keras.layers.MultiHeadAttention(
        num_heads=num_attention_heads,
        key_dim=d_model, # Size of each attention head for query Q and key K.
        dropout=dropout_rate,
        )
    # Point-wise feed-forward network.
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    # Layer normalization.
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    # Dropout for the point-wise feed-forward network.
    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    # A boolean mask.
    if mask is not None:
      mask1 = mask[:, :, None]
      mask2 = mask[:, None, :]
      attention_mask = mask1 & mask2
    else:
      attention_mask = None

    # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
    attn_output = self.mha(
        query=x,  # Query Q tensor.
        value=x,  # Value V tensor.
        key=x,  # Key K tensor.
        attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
        training=training, # A boolean indicating whether the layer should behave in training mode.
        )

    # Multi-head self-attention output after layer normalization and a residual/skip connection.
    out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

    # Point-wise feed-forward network output.
    ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
    ffn_output = self.dropout1(ffn_output, training=training)
    # Point-wise feed-forward network output after layer normalization and a residual skip connection.
    out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

    return out2

In [154]:
sample_encoder_layer = EncoderLayer(d_model=512, num_attention_heads=8, dff=2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((2, 3, 512)), training=False, mask=None)

# Print the shape.
print(sample_encoder_layer_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

(2, 3, 512)


### Define the encoder

In [263]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_vocab_size, # Input vocabulary size.
               dropout_rate=0.1
               ):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # Embeddings + Positional encoding
    self.pos_embedding = PositionalEmbedding(input_vocab_size, d_model)

    # Encoder layers.
    self.enc_layers = [
        EncoderLayer(
          d_model=d_model,
          num_attention_heads=num_attention_heads,
          dff=dff,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    # Dropout.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.flatten_layer = tf.keras.layers.GlobalAveragePooling1D()
    self.dense_layer = tf.keras.layers.Dense(256, activation='relu')
    self.final_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  # Masking.
  def compute_mask(self, x, previous_mask=None):
    return self.pos_embedding.compute_mask(x, previous_mask)

  def call(self, x, training):
    seq_len = tf.shape(x)[1]

    # Sum up embeddings and positional encoding.
    mask = self.compute_mask(x)
    x = self.pos_embedding(x)  # Shape `(batch_size, input_seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x, training=training)
    # N encoder layers.
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    # experimental thing, may not work
    x = self.flatten_layer(x)
    x = self.dense_layer(x)
    # x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = self.final_layer(x)
    return x  # Shape `(batch_size, input_seq_len, d_model)`.

In [264]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128)),
    Encoder(num_layers = 4, d_model = 128, dff = 512, num_attention_heads = 8, input_vocab_size=tokenizers.en.get_vocab_size())
])

In [265]:
input = np.random.rand(1, 128)
# print(input.shape)
# target = tf.constant([[1,2,3, 0]])

x = model((input))
x

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.38000163]], dtype=float32)>

In [266]:
model.summary()

Model: "sequential_122"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_20 (Encoder)        (None, 1)                 3569409   
                                                                 
Total params: 3,569,409
Trainable params: 3,569,409
Non-trainable params: 0
_________________________________________________________________


In [267]:
# compiling the model

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["binary_accuracy"]
)

In [271]:
features = tokenized_sentences.to_tensor()

In [272]:
my_labels = labels.to_numpy(dtype=np.float32)

In [273]:
# error!
history = model.fit(
    x=features,
    y=my_labels,
    epochs=200,
    batch_size=256,
    validation_split=0.2
)

Epoch 1/200


ValueError: ignored